<a href="https://colab.research.google.com/github/mehdihemmatyar/RecommendationSystem/blob/main/Snap_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879161 sha256=18a66b20c76a1cb1be7d431896007db34f8bd0446d4e245c15b971ad4dfbe196
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset

In [3]:
import pandas as pd

In [9]:
data = pd.read_csv("/content/drive/MyDrive/SnapUserProducts.csv")

In [5]:
# Create a Dataset object
dataset = Dataset()
dataset.fit((x[0] for x in data), (x[1] for x in data))

# Build the interaction matrix
(interactions, _) = dataset.build_interactions(((x[0], x[1]) for x in data))

# Create and train the model
model = LightFM(loss='warp')
model.fit(interactions, epochs=10)

In [6]:
# Function to get recommendations for a customer ID
def get_recommendations(customer_id, num_recommendations=5):
    user_items = interactions.tocsr()
    scores = model.predict(customer_id, np.arange(interactions.shape[1]))
    top_items = np.argsort(-scores)[:num_recommendations]
    items = dataset.mapping()[2]

    # Return the recommended Product IDs
    return [items[item_id] for item_id in top_items]


In [ ]:
# Test the recommender system
customer_id = 4130841
recommendations = get_recommendations(customer_id)

print(f"Recommended Product IDs for Customer ID {customer_id}:")
print(recommendations)

In [14]:
data.head()

,Customer ID,Product ID,rating
0,4130841,3626898,8
1,10010652,5875884,9
2,1963255,4250172,8
3,3483648,6853430,6
4,1274389,8908514,6


In [4]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095450 sha256=220208a795b06d8617d81e48b3ad65a6fbdf7cce157c2bdadf7c7b114bdec301
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [12]:
from surprise import Dataset
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from surprise import Reader


In [14]:
df = pd.DataFrame(data)
reader = Reader(rating_scale=(1, 10))
# Load the data into Surprise's Dataset format
dataset = Dataset.load_from_df(df[['Customer ID', 'Product ID', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(dataset, test_size=0.2)

# Train the SVD model on the training set
model = SVD()
model.fit(trainset)

# Make predictions for a given customer ID
customer_id = 4130841
predictions = []
for product_id in df['Product ID'].unique():
    prediction = model.predict(customer_id, product_id)
    predictions.append((product_id, prediction.est))

# Sort the predictions based on estimated ratings in descending order
predictions.sort(key=lambda x: x[1], reverse=True)

# Get the recommended product IDs
recommended_products = [product_id for product_id, _ in predictions]

print(f"Recommended products for customer {customer_id}: {recommended_products}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
# Get the top N recommended product IDs
top_n =40
recommended_products = [product_id for product_id, _ in predictions[:top_n]]

print(f"Recommended products for customer {customer_id}: {recommended_products}")

Recommended products for customer 4130841: [3626898, 5875884, 4250172, 6853430, 8908514, 3957743, 8357150, 7384289, 3882003, 4108578, 3761785, 3630066, 3917509, 8822569, 20209496, 21370132, 5286234, 9348022, 3500795, 6455005, 18041862, 7178867, 6979838, 7503235, 5937474, 4104284, 5040656, 7329064, 5312640, 21285715, 4464115, 18852157, 20355812, 9087766, 19906715, 7493300, 21964182, 7545305, 5065103, 16347923]
